## Demo of finding correspondences

- Demos generating correspondences in numpy only
- Demos same in pytorch
- Simple timing experiments in pytorch
- Demos generating non-correspondences

## Find a pixel correspondence -- first in numpy only

In [ ]:
%matplotlib inline
import sys; sys.path.append('../test'); sys.path.append('../dataset')
import numpy_correspondence_finder
import correspondence_plotter
from labelfusion import LabelFusionDataset

# from LabelFusion dataset: labelfusion.csail.mit.edu
lf = LabelFusionDataset()
scene = "2017-06-13-20"
log_dir = lf.get_full_path_for_scene(scene)
img_a_index = "0000000020"
img_b_index = "0000000400"

uv_a = (300,200)
uv_a, uv_b = numpy_correspondence_finder.find_pixel_correspondence(log_dir, img_a_index, img_b_index, uv_a=uv_a)
correspondence_plotter.plot_correspondences_from_dir(log_dir, img_a_index, img_b_index, uv_a, uv_b)

## Find a pixel correspondence -- now in PyTorch

In [ ]:
import correspondence_finder
import time
uv_a = (300,200)

img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose, 
                                                                    uv_a=uv_a)
print time.time() - start
if uv_a is not None:
    correspondence_plotter.plot_correspondences_from_dir(log_dir, img_a_index, img_b_index, uv_a, uv_b)

## PyTorch implementation -- finding many correspondences

Note that in this example, about 1/10 get pruned due to either:

1. No depth measurement in image a
2. Reprojection is outside FOV of image b
3. Occluded: the point from image a is occluded in image b
4. No depth measurement in image b (so can't be sure if occluded or not)

In [ ]:
start = time.time()
num_attempts = 50
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose, 
                                                                    num_attempts=num_attempts)
print time.time() - start, "seconds"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])
if uv_a is not None:
    correspondence_plotter.plot_correspondences_from_dir(log_dir, img_a_index, img_b_index, uv_a, uv_b)

## On CPU, how fast can we do many, many samples? (10x samples, let's not plot)

In [ ]:
num_attempts = 50000

img_a_index_2 = "0000000002"
img_b_index_2 = "0000000300"
img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index_2)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index_2)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='CPU')
print time.time() - start, "seconds 1st time"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='CPU')
print time.time() - start, "seconds 2nd time"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

img_a_index_3 = "0000000020"
img_b_index_3 = "0000000500"
img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index_3)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index_3)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='CPU')
print time.time() - start, "seconds on a new image pair"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

## On GPU, how fast?

In [ ]:
num_attempts = 50000

img_a_index_4 = "0000000003"
img_b_index_4 = "0000000301"
img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index_4)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index_4)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='GPU')
print time.time() - start, "seconds 1st time"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='GPU')
print time.time() - start, "seconds 2nd time"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

img_a_index_5 = "0000000021"
img_b_index_5 = "0000000501"
img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index_5)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index_5)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='GPU')
print time.time() - start, "seconds on a new image pair"
print "num attempted: ", num_attempts
print "num valid:     ", len(uv_a[0])

## Timing Conclusions

It's a little subtle -- cold start switching to the GPU is slow on the first image (takes a couple seconds). But on the second time on that image, the matching is quite fast, just slightly faster than the CPU (~0.018 vs 0.023 seconds).  You might think this is only because it's the same image, but no -- loading in a different image still doesn't see the 2-second cold start. (Perhaps there is pre-fetching?  Would be interested to look into it more in future.) 

Big picture though, this margin is not that large, and so without looking in deeper, computation time might be dominated by something agnostic to whether ths reprojection runs on CPU or GPU (for example, image io).

And an additional benefit is that by keeping on CPU, we will not need to take any additional precious GPU memory during training, allowing batch sizes to be higher for training.  (This process of generating correspondences happens during training, generated with each randomly selected pairs of same-scene RGBD images in the training set.)

So all in all, I am guessing this step of my pipeline will stay on CPU.  But the peace of mind is priceless knowing how fast it _could_ have been on the GPU!


### Note:

- CPU: 10-core i7-6950x
- GPU: GTX 1080 Ti
- For reference, with other simple benchmarks on this system (matrix multiply), I see up to [5,800x speedup on the GPU](https://github.com/peteflorence/matrix-multiply/blob/master/MatrixMultiply.ipynb) (at least, not accounting for initialization).

# Non-correspondences

In [ ]:
num_attempts = 3

img_a_index_2 = "0000000001"
img_b_index_2 = "0000001000"
img_a_rgb, img_a_depth, img_a_pose = lf.get_specific_rgbd_with_pose(scene, img_a_index_2)
img_b_rgb, img_b_depth, img_b_pose = lf.get_specific_rgbd_with_pose(scene, img_b_index_2)

start = time.time()
uv_a, uv_b = correspondence_finder.batch_find_pixel_correspondences(img_a_depth, img_a_pose, 
                                                                    img_b_depth, img_b_pose,
                                                                    num_attempts=num_attempts,
                                                                    device='CPU')
print time.time() - start, "seconds for matches"
print "num attempted: ", num_attempts
if uv_a is not None:
    print "num valid:     ", len(uv_a[0])

start = time.time()
uv_b_non_matches = correspondence_finder.create_non_correspondences(uv_b,num_non_matches_per_match=50)
print  time.time() - start, "seconds for non-matches"
if uv_b_non_matches is not None:
    print uv_b_non_matches[0].shape

    import torch
    # This just checks to make sure nothing is out of bounds
    print torch.min(uv_b_non_matches[0])
    print torch.min(uv_b_non_matches[1])
    print torch.max(uv_b_non_matches[0])
    print torch.max(uv_b_non_matches[1])
    
    fig, axes = correspondence_plotter.plot_correspondences_from_dir(log_dir, img_a_index_2, img_b_index_2, uv_a, uv_b, show=False)
    uv_a_long = (torch.t(uv_a[0].repeat(3, 1)).contiguous().view(-1,1), torch.t(uv_a[1].repeat(3, 1)).contiguous().view(-1,1))
    uv_b_non_matches_long = (uv_b_non_matches[0].view(-1,1), uv_b_non_matches[1].view(-1,1) )
    correspondence_plotter.plot_correspondences_from_dir(log_dir, img_a_index_2, img_b_index_2,
                                                  uv_a_long, uv_b_non_matches_long,
                                                  use_previous_plot=(fig,axes),
                                                  circ_color='r')

## Note for future:

In the plot above (at least at time of this writing), you can see a potential issue that would want a little bit of a refactor.

It is currently possible for "non-matches" to sample parts of image b for which there is no known depth.

This could be an issue for example, if it just so happens that that corner of image b matches image a.

Two things though:

1. Once I have depths from the actual projection against world model, this will be less of an issue, since there will be less holes.
2. That combined with that issue hopefully being rare, means maybe I shouldn't worry about it for now.